# Libraries

In [1]:
import volpick.data
import volpick
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import time
import datetime
from pathlib import Path
from obspy import read
from collections import namedtuple
from volpick.data.utils import freqency_index,extract_events
volpick.cache_root=Path("/home/zhongyiyuan/DATA/my_data")

In [4]:
lp_info=pd.read_csv("/home/zhongyiyuan/DATA/LP_catalog/NCEDC/LP.txt",skiprows=1)

In [5]:
lp_info

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1976/01/15 11:55:32.54,39.04767,-122.93100,19.435,1.95,Md,9,140,8,0.04,NCSN,1028378
1,1980/02/08 15:57:46.46,39.05467,-122.87383,16.550,2.45,Md,4,237,9,0.00,NCSN,1050906
2,1982/04/15 15:56:55.72,40.46300,-121.58283,18.757,2.05,Md,8,110,5,0.04,NCSN,1073742
3,1982/07/13 05:38:17.25,39.09133,-122.90000,18.841,2.06,Md,11,70,8,0.20,NCSN,1076180
4,1983/12/16 04:59:42.46,40.48617,-121.57367,16.942,1.61,Md,8,82,4,0.03,NCSN,1108516
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,2019/11/09 22:32:45.34,40.46367,-121.60917,10.420,0.33,Md,11,126,6,0.23,NCSN,73301801
1283,2019/11/30 14:02:39.18,40.49050,-121.61000,14.170,0.50,Md,15,79,3,0.34,NCSN,73311616
1284,2019/12/09 06:48:42.34,37.64333,-118.89600,-1.580,0.48,Md,18,116,0,0.13,NCSN,73312786
1285,2019/12/10 06:41:45.76,37.64933,-118.89667,-1.290,0.25,Md,15,156,1,0.07,NCSN,73313121


In [87]:
lp_info.iloc[0]["DateTime"].replace(" ","T").replace("/","-")

'1976-01-15T11:55:32.54'

In [90]:
type(lp_info.iloc[0]["Latitude"])

numpy.float64

In [34]:
summary_df=pd.read_csv("/home/zhongyiyuan/DATA/LP_catalog/NCEDC/LP.txt",skiprows=1)
summary_dict={}

In [35]:
summary_dict["ncedc1028378"]

KeyError: 'ncedc1028378'

In [37]:
"ncedc73513570" not in summary_dict

True

In [38]:
lp_info["EventID"]

0        1028378
1        1050906
2        1073742
3        1076180
4        1108516
          ...   
1282    73301801
1283    73311616
1284    73312786
1285    73313121
1286    73313791
Name: EventID, Length: 1287, dtype: int64

In [39]:
lp_info[lp_info["EventID"]==1028378]

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1976/01/15 11:55:32.54,39.04767,-122.931,19.435,1.95,Md,9,140,8,0.04,NCSN,1028378


# Load tables

In [2]:
ncedc=volpick.data.NCEDCDataset()

The default saving directory is used: /home/zhongyiyuan/DATA/my_data/ncedc
/home/zhongyiyuan/DATA/my_data/ncedc exists


In [72]:
ncedc.read(format="quakeml")

In [ ]:
ncedc

In [74]:
cat2 = ncedc.filter("time > 1997-10-14T04:31",
                  "time < 1997-10-14T04:32")

In [82]:
cat2[0].picks[3].waveform_id.id.split(".")

['NC', 'MDR', '02', 'EHZ']

In [3]:
whole_catalog_lp=ncedc.read(format="csv")
print(f"The whole catalog includes {len(whole_catalog_lp)} waveforms, where there are")
print(f""" {len(whole_catalog_lp[whole_catalog_lp["source_type"]!="lp"])} waveforms for regular earthquakes""")
print(f""" {len(whole_catalog_lp[whole_catalog_lp["source_type"]=="lp"])} waveforms for long-period earthquakes""")

The whole catalog includes 16402 waveforms, where there are
 0 waveforms for regular earthquakes
 16402 waveforms for long-period earthquakes


In [4]:
full_lp_events=extract_events(whole_catalog_lp)
print(len(full_lp_events))

1287


In [5]:
whole_catalog_lp[pd.notna(whole_catalog_lp["trace_p_max_weight"])|pd.notna(whole_catalog_lp["trace_s_max_weight"])]

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_depth_km,source_magnitude,source_magnitude_type,source_type,station_network_code,station_code,station_location_code,trace_channel,trace_p_arrival_time,trace_s_arrival_time,trace_p_max_weight,trace_s_max_weight,trace_p_first_motion
0,ncedc1028378,1976-01-15T11:55:32.540000Z,39.047667,-122.931000,19.43,1.95,Md,lp,NC,GCV,01,EH,1976-01-15T11:55:39.080000Z,NaN,1.64,NaN,NaN
2,ncedc1028378,1976-01-15T11:55:32.540000Z,39.047667,-122.931000,19.43,1.95,Md,lp,NC,GGL,--,EH,1976-01-15T11:55:37.790000Z,NaN,0.82,NaN,NaN
3,ncedc1028378,1976-01-15T11:55:32.540000Z,39.047667,-122.931000,19.43,1.95,Md,lp,NC,GHG,--,EH,1976-01-15T11:55:36.480000Z,NaN,1.64,NaN,U
4,ncedc1028378,1976-01-15T11:55:32.540000Z,39.047667,-122.931000,19.43,1.95,Md,lp,NC,GHL,--,EH,1976-01-15T11:55:36.030000Z,NaN,1.64,NaN,NaN
5,ncedc1028378,1976-01-15T11:55:32.540000Z,39.047667,-122.931000,19.43,1.95,Md,lp,NC,GMC,--,EH,1976-01-15T11:55:38.950000Z,NaN,0.32,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16397,ncedc73313791,2019-12-12T05:44:22.330000Z,40.543333,-121.662667,16.86,0.72,Md,lp,NC,LDH,--,HH,2019-12-12T05:44:25.920000Z,2019-12-12T05:44:28.440000Z,0.28,1.41,NaN
16398,ncedc73313791,2019-12-12T05:44:22.330000Z,40.543333,-121.662667,16.86,0.72,Md,lp,NC,LEL,--,EH,2019-12-12T05:44:26.400000Z,2019-12-12T05:44:28.770000Z,1.41,0.04,U
16399,ncedc73313791,2019-12-12T05:44:22.330000Z,40.543333,-121.662667,16.86,0.72,Md,lp,NC,LRD,--,EH,2019-12-12T05:44:26.680000Z,2019-12-12T05:44:30.910000Z,0.28,1.41,NaN
16400,ncedc73313791,2019-12-12T05:44:22.330000Z,40.543333,-121.662667,16.86,0.72,Md,lp,NC,LRR,--,EH,2019-12-12T05:44:26.310000Z,2019-12-12T05:44:28.500000Z,1.41,1.41,U


In [20]:
zero_weight_subcatalog=whole_catalog_lp[pd.isna(whole_catalog_lp["trace_p_max_weight"])&pd.isna(whole_catalog_lp["trace_s_max_weight"])]
print(f"There are {len(zero_weight_subcatalog)} waveforms whose arrival weights are 0")

There are 2891 waveforms whose arrival weights are 0


In [58]:
len_both_S_and_S=len(whole_catalog_lp[pd.notna(whole_catalog_lp["trace_p_arrival_time"])&pd.notna(whole_catalog["trace_s_arrival_time"])&pd.notna(whole_catalog_lp["trace_p_max_weight"])&pd.notna(whole_catalog_lp["trace_s_max_weight"])])
len_both_S_and_S_rg=len(whole_catalog_lp[pd.notna(whole_catalog_lp["trace_p_arrival_time"])&pd.notna(whole_catalog_lp["trace_s_arrival_time"])&pd.notna(whole_catalog_lp["trace_p_max_weight"])&pd.notna(whole_catalog_lp["trace_s_max_weight"])&(whole_catalog_lp["source_type"]!="lp")])
len_both_S_and_S_lp=len(whole_catalog_lp[pd.notna(whole_catalog_lp["trace_p_arrival_time"])&pd.notna(whole_catalog_lp["trace_s_arrival_time"])&pd.notna(whole_catalog_lp["trace_p_max_weight"])&pd.notna(whole_catalog_lp["trace_s_max_weight"])&(whole_catalog_lp["source_type"]=="lp")])
print(f"{len_both_S_and_S} waveforms have both P and S arrivals with nonzero weights")
print(f"including {len_both_S_and_S_rg} regular earthquake waveforms")
print(f"and {len_both_S_and_S_lp} long-period earthquake waveforms")

5422 waveforms have both P and S arrivals with nonzero weights
including 0 regular earthquake waveforms
and 5422 long-period earthquake waveforms


In [60]:
len(whole_catalog_lp[pd.notna(whole_catalog_lp["trace_p_arrival_time"])&pd.notna(whole_catalog["trace_s_arrival_time"])&pd.notna(whole_catalog_lp["trace_p_max_weight"])&pd.notna(whole_catalog_lp["trace_s_max_weight"])].drop_duplicates(subset="source_id",keep="first",inplace=False))

1140

In [91]:
whole_catalog_lp["station_location_code"]

0        01
1        --
2        --
3        --
4        --
         ..
16397    --
16398    --
16399    --
16400    --
16401    --
Name: station_location_code, Length: 16402, dtype: object

In [6]:
same_station_catalog=whole_catalog_lp[whole_catalog_lp.duplicated(subset=["source_id","station_network_code","station_code"],keep=False)]
same_station_catalog.to_csv("same_station_ncedc.csv",index=False)

In [7]:
len(same_station_catalog)

336

In [8]:
whole_event_table=whole_catalog_lp.drop_duplicates(subset="source_id",keep="first",ignore_index=True).copy()
print(f"{len(whole_event_table)} events")

1287 events


In [9]:
# check completeness
whole_catalog_with_P_S=whole_catalog_lp[ (pd.notna(whole_catalog_lp["trace_s_arrival_time"])) & (pd.notna(whole_catalog_lp["trace_p_arrival_time"])) ]

download_table=pd.read_csv(ncedc.save_dir/"mseed_log"/"downloads.csv")
failed_downloads=pd.read_csv(ncedc.save_dir/"mseed_log"/"failed_downloads.csv")
abnormal_traces=pd.read_csv(ncedc.save_dir/"mseed_log"/"abnormal_traces.csv")

len(download_table)+len(failed_downloads)+len(abnormal_traces)==len(whole_catalog_with_P_S)

True

In [10]:
len(whole_catalog_with_P_S)

5422

# Downloading log

In [6]:
download_table=pd.read_csv(ncedc.save_dir/"mseed_log"/"downloads.csv")
print(f"{len(download_table)} seismograms have been downloaded sucessfully.")
print(f""" {len(download_table[download_table["source_type"]!="lp"])} waveforms of regular earthquakes""")
print(f""" {len(download_table[download_table["source_type"]=="lp"])} waveforms of long-period earthquakes""")

4856 seismograms have been downloaded sucessfully.
 0 waveforms of regular earthquakes
 4856 waveforms of long-period earthquakes


In [24]:
x=ncedc.save_dir/"mseed"

In [30]:
x

PosixPath('/home/zhongyiyuan/DATA/my_data/ncedc/mseed')

In [29]:
x.parent/f"{x.name}_log"

PosixPath('/home/zhongyiyuan/DATA/my_data/ncedc/mseed_log')

In [27]:
ncedc.save_dir/f"{x.name}_log"

PosixPath('/home/zhongyiyuan/DATA/my_data/ncedc/mseed_log')

60

In [58]:
event_table=download_table.drop_duplicates(subset="source_id",keep="first",ignore_index=True).copy()

In [60]:
print(f"{len(event_table)} events")

1108 events


In [43]:
download_table[["station_network_code","station_code","station_location_code","trace_channel"]]

,station_network_code,station_code,station_location_code,trace_channel
0,BK,MIN,N1,EH
1,NC,LHC,--,EH
2,NC,LHK,--,EH
3,NC,LMZ,--,EH
4,NC,LSL,--,EH
...,...,...,...,...
6179,BK,DANT,00,HH
6180,NC,LDH,--,HH
6181,NC,LEL,--,EH
6182,NC,LRD,--,EH


# Failed downloads

In [25]:
failed_downloads=pd.read_csv(ncedc.save_dir/"mseed_log"/"failed_downloads.csv")

In [26]:
solvable_errors=failed_downloads[~failed_downloads["Error"].isin(["FDSNNoDataException"])]
nodata_errors=failed_downloads[failed_downloads["Error"]=="FDSNNoDataException"]
print(f"{len(solvable_errors)} failed downloads are not due to FDSNNoDataException")
print(f"{len(nodata_errors)} FDSNNoDataException")

0 failed downloads are not due to FDSNNoDataException
225 FDSNNoDataException


In [46]:
solvable_errors

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_depth_km,source_magnitude,source_magnitude_type,source_type,station_network_code,station_code,...,trace_p_min_weight,trace_s_min_weight,trace_p_first_motion,station_latitude_deg,station_longitude_deg,station_elevation_m,station_epicentral_distance_m,path_azimuth_deg,path_back_azimuth_deg,Error


# Abnormal traces

In [27]:
abnormal_traces=pd.read_csv(ncedc.save_dir/"mseed_log"/"abnormal_traces.csv")

In [41]:
not_in_inventory=abnormal_traces[abnormal_traces["remark"]=="not_in_inventory"][["station_network_code","station_code","station_location_code","trace_channel", "trace_p_arrival_time", "trace_s_arrival_time"]]
no_waveforms=abnormal_traces[abnormal_traces["remark"]=="No_waveforms"][["station_network_code","station_code","station_location_code","trace_channel", "trace_p_arrival_time", "trace_s_arrival_time"]]
p_lt_origin=abnormal_traces[abnormal_traces["remark"]=="P<origin"][["station_network_code","station_code","station_location_code","trace_channel", "trace_p_arrival_time", "trace_s_arrival_time"]]
phases_in_gap=abnormal_traces[abnormal_traces["remark"]=="phases_in_gap"][["station_network_code","station_code","station_location_code","trace_channel", "trace_p_arrival_time", "trace_s_arrival_time"]]

In [45]:
print(len(not_in_inventory))
print(len(no_waveforms))
print(len(p_lt_origin))
print(len(phases_in_gap))

0
304
8
29


In [43]:
print(len(no_waveforms))

304


In [44]:
len(p_lt_origin)

8

In [30]:
len(abnormal_traces)

341

In [32]:
np.unique(abnormal_traces["remark"].to_numpy())

array(['No_waveforms', 'P<origin', 'phases_in_gap'], dtype=object)

In [16]:
from obspy.clients.fdsn import RoutingClient
client = RoutingClient("iris-federator")

In [30]:
client=Client("IRIS")
client.get_stations(network="BK",station="CMB",location="--")

Inventory created at 2023-06-12T09:43:22.224700Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?network=BK&station=C...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			BK
		Stations (2):
			BK.CMB (Columbia College, Columbia, CA, USA) (2x)
		Channels (0):


In [29]:
client=Client("NCEDC")
client.get_stations(network="BK",station="CMB",location="--")

Inventory created at 2023-06-12T02:42:52.000000Z
	Created by: NCEDC WEB SERVICE: fdsnws-station | version: 1.2
		    http://service.ncedc.org/fdsnws/station/1/query?net=BK&sta=CMB&leve...
	Sending institution: NCEDC (NCEDC)
	Contains:
		Networks (1):
			BK
		Stations (2):
			BK.CMB (Columbia College, Columbia, CA, USA) (2x)
		Channels (0):


In [21]:
client.get_stations(network="WR",station="MGL",location="N1")

Inventory created at 2023-06-12T09:28:36.312326Z
	Created by: ObsPy 1.4.0
		    https://www.obspy.org
	Sending institution: NCEDC,ObsPy FDSN Routing 1.4.0,SCEDC (NCEDC,SCEDC)
	Contains:
		Networks (2):
			WR (2x)
		Stations (2):
			WR.MGL (Magalia) (2x)
		Channels (0):


In [22]:
client.get_stations(network="AK",station="SDG",location="--")

FDSNException: Unknown HTTP code: 404
Detailed response of server:

 -- Error 404: No data matched request.
Request:
http://service.iris.edu/irisws/fedcatalog/1/query
Request Submitted:
2023/06/12 09:32:46 UTC
Service version:
Service: irisws-fedcatalog  version: 1.1.17

# VT data

In [7]:
ncedc_vt=volpick.data.NCEDCDataset(root_folder_name="ncedc_vt", cat_file_name="ncedc_catalog_vt")

The default saving directory is used: /home/zhongyiyuan/DATA/my_data/ncedc_vt
/home/zhongyiyuan/DATA/my_data/ncedc_vt exists


In [9]:
whole_catalog_rg=ncedc_vt.read(format="csv")
print(len(whole_catalog_rg))
contained_lp=whole_catalog_rg[whole_catalog_rg["source_id"].isin(whole_catalog_lp["source_id"])]
print(f"Exclude {len(contained_lp)} lp waveforms from the catalog")
whole_catalog_rg=whole_catalog_rg[~whole_catalog_rg["source_id"].isin(whole_catalog_lp["source_id"])]
print(f"The whole catalog includes {len(whole_catalog_rg)} waveforms, where there are")
print(f""" {len(whole_catalog_rg[whole_catalog_rg["source_type"]!="lp"])} waveforms of regular earthquakes""")
print(f""" {len(whole_catalog_rg[whole_catalog_rg["source_type"]=="lp"])} waveforms of long-period earthquakes""")

4519837
Exclude 5214 lp waveforms from the catalog
The whole catalog includes 4514623 waveforms, where there are
 4514623 waveforms of regular earthquakes
 0 waveforms of long-period earthquakes


In [39]:
x=extract_events(contained_lp)

In [10]:
whole_catalog_rg["source_type"]

0          e
1          e
2          e
3          e
4          e
          ..
4519832    e
4519833    e
4519834    e
4519835    e
4519836    e
Name: source_type, Length: 4514623, dtype: object

In [11]:
traces_with_P_and_S=whole_catalog_rg[(pd.notna(whole_catalog_rg["trace_p_arrival_time"]))&(pd.notna(whole_catalog_rg["trace_s_arrival_time"]))]

In [12]:
regular_events_with_P_and_S=extract_events(traces_with_P_and_S) #events in the region

In [13]:
regular_events_with_P_and_S

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_depth_km,source_magnitude,source_magnitude_type,source_type
0,ncedc71328725,2010-01-01T02:41:04.750000Z,39.388667,-121.248167,17.98,1.49,Md,e
1,ncedc71328740,2010-01-01T03:10:33.280000Z,38.789167,-122.776500,0.35,0.17,Md,e
2,ncedc71328775,2010-01-01T04:17:35.660000Z,38.787167,-122.770833,1.35,1.51,Md,e
3,ncedc71328790,2010-01-01T05:17:37.280000Z,37.441000,-118.898667,2.62,0.76,Md,e
4,ncedc71328795,2010-01-01T05:25:27.130000Z,37.360000,-121.724000,5.67,1.00,Md,e
...,...,...,...,...,...,...,...,...
101009,ncedc73313206,2019-12-10T14:58:35.510000Z,37.483000,-118.822333,4.83,0.70,Md,e
101010,ncedc73313216,2019-12-10T16:23:17.650000Z,38.801333,-122.823167,2.63,0.26,Md,e
101011,ncedc73313316,2019-12-10T20:24:37.230000Z,37.259833,-121.639500,2.87,1.12,Md,e
101012,ncedc73313351,2019-12-10T22:55:22.580000Z,37.479833,-118.857500,1.59,0.78,Md,e


In [14]:
from obspy.geodetics import gps2dist_azimuth
def distance_to_volcano(ev_lat,ev_lon,vol_lat,vol_lon):
    dist, _, _ = gps2dist_azimuth(
        ev_lat,
        ev_lon,
        vol_lat,
        vol_lon,
    )
    return dist

In [15]:
vol=pd.read_csv("/mnt/DATA2/YiyuanZhong/my_data/california_volcanoes/california_vol.csv")

In [16]:
region_mask=pd.Series([False]*len(regular_events_with_P_and_S))

vol_dist_thre=20e3
for lat, lon in zip(vol["lat"],vol["lon"]):
    def check_dist(x):
        if np.abs(x["source_latitude_deg"]-lat)>0.7 or np.abs(x["source_longitude_deg"]-lon)>0.7:
            return False
        else:
            return distance_to_volcano(x["source_latitude_deg"],x["source_longitude_deg"],lat,lon)<vol_dist_thre
        
    dist_mask=regular_events_with_P_and_S.apply(check_dist, axis=1)
    region_mask=region_mask|dist_mask

In [18]:
# downsampled events
events_near_volcanoes=regular_events_with_P_and_S[region_mask]

In [19]:
events_near_volcanoes

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_depth_km,source_magnitude,source_magnitude_type,source_type
6,ncedc71328810,2010-01-01T06:26:08.680000Z,38.816667,-122.792000,1.84,1.32,Md,e
7,ncedc71328835,2010-01-01T08:23:50.080000Z,38.815167,-122.804167,1.49,1.31,Md,e
8,ncedc71328855,2010-01-01T10:33:07.180000Z,37.621333,-119.041000,2.41,0.98,Md,e
9,ncedc71328860,2010-01-01T11:00:08.460000Z,38.796500,-122.740333,1.42,1.65,Md,e
11,ncedc71328910,2010-01-01T14:04:40.340000Z,38.816333,-122.819167,3.03,0.25,Md,e
...,...,...,...,...,...,...,...,...
101002,ncedc73313031,2019-12-09T22:08:56.080000Z,37.555500,-118.832500,3.59,0.78,Md,e
101003,ncedc73313146,2019-12-10T08:37:13.940000Z,37.557000,-118.833833,2.92,0.79,Md,e
101005,ncedc73313166,2019-12-10T09:48:01.740000Z,37.545000,-118.858667,6.49,0.31,Md,e
101008,ncedc73313186,2019-12-10T14:02:55.080000Z,38.791667,-122.740167,2.15,1.39,Md,e


In [57]:
np.savetxt(fname="nc_rg.txt",
           X=regular_events_with_P_and_S[["source_longitude_deg","source_latitude_deg"]].values,
           fmt='%-15.9f')
np.savetxt(fname="nc_vt.txt",
           X=events_near_volcanoes[["source_longitude_deg","source_latitude_deg"]].values,
           fmt='%-15.9f')

In [20]:
filter_mask=np.zeros(len(events_near_volcanoes),dtype=bool)
print(len(events_near_volcanoes))
print(len(filter_mask))
n_events=5000
rand_idxs = np.sort(
    np.random.default_rng(seed=100).choice(
        len(events_near_volcanoes), size=n_events, replace=False
    )
)
filter_mask[rand_idxs] = True
downsampled_events_near_volcanoes=events_near_volcanoes[filter_mask]

61249
61249


In [21]:
downsampled_vt_traces=traces_with_P_and_S[traces_with_P_and_S["source_id"].isin(downsampled_events_near_volcanoes["source_id"])]

In [71]:
np.savetxt(fname="nc_vt_ds.txt",
           X=downsampled_events_near_volcanoes[["source_longitude_deg","source_latitude_deg"]].values,
           fmt='%-15.9f')

In [22]:
downsampled_vt_traces.reset_index(drop=True,inplace=True)

In [24]:
downsampled_vt_traces.to_csv("/mnt/DATA2/YiyuanZhong/my_data/ncedc_vt/downsampled_ncedc_vt_with_PS.csv",index=False)

In [26]:
download_table_vt=pd.read_csv("/mnt/DATA2/YiyuanZhong/my_data/ncedc_vt/mseed_log/downloads.csv")
print(f"{len(download_table_vt)} seismograms have been downloaded sucessfully.")
print(f""" {len(download_table_vt[download_table_vt["source_type"]!="lp"])} waveforms of regular earthquakes""")
print(f""" {len(download_table_vt[download_table_vt["source_type"]=="lp"])} waveforms of long-period earthquakes""")

17191 seismograms have been downloaded sucessfully.
 17191 waveforms of regular earthquakes
 0 waveforms of long-period earthquakes


In [27]:
dl_e=extract_events(download_table_vt)

In [28]:
len(dl_e)

4814

In [30]:
17191/4814*1094

3906.7208142916493